This notebook was used to perform training and evaluation of a simple two-hidden-layer and three-hidden layers on
MNIST and Fashion MNIST datasets.

In [37]:
from typing import List, Tuple
import torch.nn as nn
import torch
import os
import torchvision
import numpy as np
import pandas as pd
from sklearn import metrics

In [38]:
# Set seed for consistent behavior (though different results will still occur due to computation on GPU - if present)
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

# Set the device for computation
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [39]:
# Set data type for computations in the neural network
dtype = torch.float

In [40]:
# Function to convert the inputs into spikes
def convert_to_firing_time(x, tau=20, thr=0.2, tmax=1.0, epsilon=1e-7):
    idx = x < thr
    x = np.clip(x, thr + epsilon, 1e9)
    T = tau * np.log(x / (x - thr))
    T[idx] = tmax
    return T

# This function is used to generate features and labels for the simulator in batches
def sparse_data_generator(x, y, batch_size, num_steps, num_units, time_step=1e-3, shuffle=True):

    labels_ = np.array(y, dtype=np.int)
    number_of_batches = len(x) // batch_size
    sample_index = np.arange(len(x))

    # compute discrete firing times
    tau_eff = 20e-3 / time_step
    firing_times = np.array(convert_to_firing_time(x, tau=tau_eff, tmax=num_steps), dtype=np.int)
    unit_numbers = np.arange(num_units)

    # shuffle if set to True
    if shuffle:
        np.random.shuffle(sample_index)

    counter = 0
    while counter < number_of_batches:
        batch_index = sample_index[batch_size * counter:batch_size * (counter + 1)]

        coo = [[] for _ in range(3)]
        for bc, idx in enumerate(batch_index):
            c = firing_times[idx] < num_steps
            times, units = firing_times[idx][c], unit_numbers[c]

            batch = [bc for _ in range(len(times))]
            coo[0].extend(batch)
            coo[1].extend(times)
            coo[2].extend(units)

        i = torch.LongTensor(coo).to(device)
        v = torch.FloatTensor(np.ones(len(coo[0]))).to(device)

        X_batch = torch.sparse.FloatTensor(i, v, torch.Size([batch_size, num_steps, num_units])).to(device)
        y_batch = torch.tensor(labels_[batch_index], device=device, dtype=torch.long)

        yield X_batch.to(device=device), y_batch.to(device=device)

        counter += 1


class SurrGradSpike(torch.autograd.Function):
    scale = 100.0  # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input / (SurrGradSpike.scale * torch.abs(input) + 1.0) ** 2
        return grad

In [41]:
class DeepSNNModel:
    """
    This class implements a simple deep snn model that comprises a set of fully connected layers
    """

    def __init__(self, units: List[int], weight_scale: Tuple = (7.0, 1.0), recurrent=False, time_step=1e-3,
                 tau_mem=10e-3,
                 tau_syn=5e-3):
        """
        :param units: list of units in each layer
        :param weight_scale: tuple of one or two values, controls the scaling of the weights
        :param recurrent: whether the network is recurrent one - additional recurrent weights are used
        :param time_step: how long does one time step take (seconds)
        :param tau_mem: membrane tau parameter
        :param tau_syn: synapse tau parameter
        """
        self.units = units
        self.tau_mem = tau_mem
        self.tau_syn = tau_syn
        self.alpha = float(np.exp(-time_step / tau_syn))
        self.beta = float(np.exp(-time_step / tau_mem))
        self.is_recurrent = recurrent

        if len(weight_scale) == 2:
            self.weight_scale = weight_scale[0] * (weight_scale[1] - self.beta)
        else:
            self.weight_scale = weight_scale[0]
        self.weights = self._init_weights()
        self.recurrent_weights = None if not recurrent else self._init_recurrent_weights()

    def _init_weights(self):
        """
        Initializes weights between layers
        :return: list of torch tensors representing weights
        """
        weights = []
        for i in range(len(self.units) - 1):
            wi = torch.empty((self.units[i], self.units[i + 1]), device=device, dtype=dtype, requires_grad=True)
            torch.nn.init.normal_(wi, mean=0.0, std=self.weight_scale / np.sqrt(self.units[i]))
            weights.append(wi)
        return weights

    def _init_recurrent_weights(self):
        """
        Initializes recurrent weights for recurrent network for every hidden layer
        :return: list of torch tensors representing recurrent weights
        """
        recurrent_weights = []
        for i in range(1, len(self.units) - 1):
            vi = torch.empty((self.units[i], self.units[i]), device=device, dtype=dtype, requires_grad=True)
            torch.nn.init.normal_(vi, mean=0.0, std=self.weight_scale / np.sqrt(self.units[i]))
            recurrent_weights.append(vi)
        return recurrent_weights

    def run_recurrent(self, inputs, batch_size, steps, spike_fn=SurrGradSpike.apply):
        """
        Runs recurrent network
        :param inputs: input data
        :param batch_size: batch size
        :param steps: number of timesteps
        :param spike_fn: surrogate gradient function
        :return: output layer results
        """
        layer_outputs = [inputs]
        mem_recs = []
        spike_recs = []

        # compute activity in each hidden layer
        for i in range(1, len(self.units) - 1):  # the current hidden layer index
            hidden_units = self.units[i]  # neurons in the current hidden layer
            syn_hidden_i = torch.zeros((batch_size, hidden_units), device=device, dtype=dtype)
            mem_hidden_i = torch.zeros((batch_size, hidden_units), device=device, dtype=dtype)

            mem_rec_hidden_i = [mem_hidden_i]
            spike_rec_hidden_i = [mem_hidden_i]

            hi = torch.zeros((batch_size, hidden_units), device=device, dtype=dtype)
            hi_from_prev_layer = torch.einsum('abc, cd -> abd', (layer_outputs[i - 1], self.weights[i - 1]))

            for dt in range(steps):
                mem_threshold = mem_hidden_i - 1.0
                spike_out = spike_fn(mem_threshold)
                rst = torch.zeros_like(mem_hidden_i)
                c = (mem_threshold > 0)
                rst[c] = torch.ones_like(mem_hidden_i)[c]

                hi = hi_from_prev_layer[:, dt] + torch.einsum('ab, bc -> ac', (hi, self.recurrent_weights[i - 1]))
                new_syn = self.alpha * syn_hidden_i + hi
                new_mem = self.beta * mem_hidden_i + syn_hidden_i - rst

                mem_hidden_i = new_mem
                syn_hidden_i = new_syn

                mem_rec_hidden_i.append(mem_hidden_i)
                spike_rec_hidden_i.append(spike_out)

            spike_rec_hidden_i = torch.stack(spike_rec_hidden_i, dim=1)
            mem_rec_hidden_i = torch.stack(mem_rec_hidden_i, dim=1)

            layer_outputs.append(spike_rec_hidden_i)  # append output so it can be fed to the next hidden layer
            mem_recs.append(mem_rec_hidden_i)
            spike_recs.append(spike_rec_hidden_i)

        # output layer
        hn = torch.einsum('abc, cd -> abd', (layer_outputs[-1], self.weights[-1]))
        flt = torch.zeros((batch_size, self.units[-1]), device=device, dtype=dtype)
        spike_out = torch.zeros((batch_size, self.units[-1]), device=device, dtype=dtype)

        out_rec = [spike_out]
        for dt in range(steps):
            new_flt = self.alpha * flt + hn[:, dt]
            new_out = self.beta * spike_out + flt

            flt = new_flt
            spike_out = new_out

            out_rec.append(spike_out)

        out_rec = torch.stack(out_rec, dim=1)
        return out_rec, spike_recs, layer_outputs, mem_recs

    def run_feed_forward(self, inputs, batch_size, steps, spike_fn=SurrGradSpike.apply):
        """
        Run feed forward network in the same manner as a recurrent one except that no recurrent weights are calculated
        :param inputs: input data
        :param batch_size: batch size
        :param steps: number of time steps
        :param spike_fn: surrogate gradient function
        :return: output layer results
        """
        layer_outputs = [inputs]
        mem_recs = []
        spike_recs = []

        # compute activity of every hidden layer
        # hidden layers are stored from index 1 to index len(self.units) - 2
        for i in range(1, len(self.units) - 1):
            hidden_units = self.units[i]  # units in next layer

            # sum of output from previous layer and weights of current hidden layer
            hi = torch.einsum('abc,cd->abd', (layer_outputs[i - 1], self.weights[i - 1]))
            syn_hidden_i = torch.zeros((batch_size, hidden_units), device=device, dtype=dtype)  # synapses
            mem_hidden_i = torch.zeros((batch_size, hidden_units), device=device, dtype=dtype)  # membranes

            mem_rec_hidden_i = [mem_hidden_i]
            spike_rec_hidden_i = [mem_hidden_i]

            for dt in range(steps):
                mem_threshold = mem_hidden_i - 1.0
                spike_out = spike_fn(mem_threshold)
                rst = torch.zeros_like(mem_hidden_i)
                c = (mem_threshold > 0)
                rst[c] = torch.ones_like(mem_hidden_i)[c]

                new_syn = self.alpha * syn_hidden_i + hi[:, dt]
                new_mem = self.beta * mem_hidden_i + syn_hidden_i - rst
                mem_hidden_i = new_mem
                syn_hidden_i = new_syn

                mem_rec_hidden_i.append(mem_hidden_i)
                spike_rec_hidden_i.append(spike_out)

            spike_rec_hidden_i = torch.stack(spike_rec_hidden_i, dim=1)
            mem_rec_hidden_i = torch.stack(mem_rec_hidden_i, dim=1)

            layer_outputs.append(spike_rec_hidden_i)  # append output so it can be fed to the next hidden layer
            mem_recs.append(mem_rec_hidden_i)
            spike_recs.append(spike_rec_hidden_i)

        # readout layer
        hn = torch.einsum('abc,cd->abd', (layer_outputs[-1], self.weights[-1]))
        flt = torch.zeros((batch_size, self.units[-1]), device=device, dtype=dtype)
        spike_out = torch.zeros((batch_size, self.units[-1]), device=device, dtype=dtype)

        out_rec = [spike_out]
        for dt in range(steps):
            new_flt = self.alpha * flt + hn[:, dt]
            new_out = self.beta * spike_out + flt
            flt = new_flt
            spike_out = new_out
            out_rec.append(spike_out)

        out_rec = torch.stack(out_rec, dim=1)
        return out_rec, spike_recs, layer_outputs, mem_recs

    def train(self, x_data, y_data, batch_size, num_steps=100, time_step=1e-3, lr=1e-3, num_epochs=10):
        """
        Train the network
        :param x_data: features (training data)
        :param y_data: labels (training)
        :param batch_size: number of elements per batch
        :param num_steps: number of time steps
        :param time_step: how long does a time step take (s)
        :param lr: learning rate
        :param num_epochs: number of epochs for training
        :return:
        """

        # Create optimizer, and loss function
        optimizer = torch.optim.Adam(self.weights, lr=lr, betas=(0.9, 0.999))
        log_softmax_fn = nn.LogSoftmax(dim=1)
        loss_fn = nn.NLLLoss()

        # Begin training
        loss_hist = []
        for epoch in range(num_epochs):
            local_loss = []
            for x_local, y_local in sparse_data_generator(x_data, y_data, batch_size, num_steps, self.units[0],
                                                          time_step):

                if not self.is_recurrent:  # if the network does not contain recurrent weights run it as feedforward
                    output, spike_recs, layer_outputs, mem_recs = self.run_feed_forward(x_local.to_dense(), batch_size,
                                                                                        num_steps)
                else:
                    output, spike_recs, layer_outputs, mem_recs = self.run_recurrent(x_local.to_dense(), batch_size,
                                                                                     num_steps)

                output_max, _ = torch.max(output, 1)
                log_p_y = log_softmax_fn(output_max) # apply log softmax function

                loss_val = loss_fn(log_p_y, y_local) # calculate the loss

                # Perform the optimization
                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()
                local_loss.append(loss_val.item())

            mean_loss = np.mean(local_loss)
            print('Epoch {}: loss={:.5f}'.format(epoch + 1, mean_loss)) # log the loss to console
            loss_hist.append(mean_loss)

        return loss_hist

In [42]:
def get_metrics(x_test, y_test, batch_size, snn_model, time_step=1e-3, num_steps=100):
    """
    Calculate metrics for the network
    :param x_test: testing features
    :param y_test: testing labels
    :param batch_size: number of elements per batch
    :param snn_model: reference to the trained snn model
    :param time_step: how long does a time step take (s)
    :param num_steps: number of time steps
    :return: accuracy, precision, recall, f1 metric and confusion matrix
    """
    predictions = []

    # Truncate the remaining number of samples according to batch size
    samples = (x_test.shape[0] // batch_size ) * batch_size
    x_test, y_test = x_test[:samples], y_test[:samples]
    for x_local, y_local in sparse_data_generator(x_test, y_test, batch_size, num_steps, snn_model.units[0], time_step,
                                                  False):

        # Run feedforward/recurrent network and get the output
        output, spike_recs, layer_outputs, mem_recs = snn_model.run_recurrent(x_local.to_dense(), batch_size, num_steps) \
            if snn_model.recurrent_weights else snn_model.run_feed_forward(x_local.to_dense(), batch_size, num_steps)

        output_max, _ = torch.max(output, 1)
        _, output_argmax = torch.max(output_max, 1)

        # Append the current batch
        predictions.append(output_argmax)

    # Concatenate to a single tensor
    predictions = torch.cat(predictions)

    # Map to numpy
    predictions = predictions.cpu().detach().numpy()

    # Now calculate the metrics
    # Use the "micro" average - i.e. the calculation is done globally, counting total number of tp, fp and fn for
    # each class
    precision = metrics.precision_score(y_true=y_test, y_pred=predictions, average='macro')
    recall = metrics.recall_score(y_true=y_test, y_pred=predictions, average='macro')
    f1 = metrics.f1_score(y_true=y_test, y_pred=predictions, average='macro')
    accuracy = metrics.accuracy_score(y_true=y_test, y_pred=predictions)
    confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=predictions)

    # Return the metrics
    return accuracy, precision, recall, f1, confusion_matrix


In [43]:
# Download datasets through PyTorch (if necessary)
dataset_folder = os.path.join('cached_datasets')

train_dataset = torchvision.datasets.MNIST(dataset_folder, train=True,
                                           transform=None, target_transform=None, download=True)
test_dataset = torchvision.datasets.MNIST(dataset_folder, train=False,
                                          transform=None, target_transform=None, download=True)

In [44]:
# Declaration of models that will be tested on the MNIST and the Fashion MNIST datasets
models_mnist = [
    DeepSNNModel([28*28, 256, 128, 10]),
    DeepSNNModel([28*28, 256, 128, 64, 10]),
    DeepSNNModel([28*28, 256, 128, 10], recurrent=True),
    DeepSNNModel([28*28, 256, 128, 64, 10], recurrent=True)
]

# Standardize the MNIST dataset
mnist_x_train = np.array(train_dataset.data, dtype=np.float)
mnist_x_train = mnist_x_train.reshape(mnist_x_train.shape[0], -1) / 255
mnist_x_test = np.array(test_dataset.data, dtype=np.float)
mnist_x_test = mnist_x_test.reshape(mnist_x_test.shape[0], -1) / 255

mnist_y_train = np.array(train_dataset.targets, dtype=np.int)
mnist_y_test = np.array(test_dataset.targets, dtype=np.int)

'The MNIST dataset is ready.'

'The MNIST dataset is ready.'

In [45]:
print('Running benchmark on MNIST dataset')

# Array for statistics of each model
mnist_stats = []

# Run each model
for i, model in enumerate(models_mnist):
    model.train(mnist_x_train, mnist_y_train, 256, num_epochs=30)
    accuracy, precision, recall, f1, confusion_matrix = get_metrics(mnist_x_test, mnist_y_test, 256, model)
    stats = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': confusion_matrix
    }

    print(f'Model {i+1}: {stats}')
    mnist_stats.append(stats)


Running benchmark on MNIST dataset
Epoch 1: loss=0.41898
Epoch 2: loss=0.17241
Epoch 3: loss=0.13497
Epoch 4: loss=0.11038
Epoch 5: loss=0.10026
Epoch 6: loss=0.08224
Epoch 7: loss=0.07420
Epoch 8: loss=0.06370
Epoch 9: loss=0.06477
Epoch 10: loss=0.05850
Epoch 11: loss=0.05357
Epoch 12: loss=0.04823
Epoch 13: loss=0.04377
Epoch 14: loss=0.04931
Epoch 15: loss=0.04287
Epoch 16: loss=0.03852
Epoch 17: loss=0.03905
Epoch 18: loss=0.03719
Epoch 19: loss=0.04059
Epoch 20: loss=0.03788
Epoch 21: loss=0.02963
Epoch 22: loss=0.03222
Epoch 23: loss=0.03603
Epoch 24: loss=0.03475
Epoch 25: loss=0.02788
Epoch 26: loss=0.02541
Epoch 27: loss=0.02777
Epoch 28: loss=0.03440
Epoch 29: loss=0.03188
Epoch 30: loss=0.03120
Model 0: {'accuracy': 0.9697516025641025, 'precision': 0.9697470178734366, 'recall': 0.9695371760516318, 'f1': 0.9694956147153926, 'confusion_matrix': array([[ 972,    0,    0,    2,    0,    0,    1,    0,    2,    2],
       [   3, 1115,    3,    1,    2,    1,    4,    1,    3,   

In [46]:
# Fashion MNIST
train_dataset = torchvision.datasets.FashionMNIST(dataset_folder, train=True,
                                           transform=None, target_transform=None, download=True)
test_dataset = torchvision.datasets.FashionMNIST(dataset_folder, train=False,
                                          transform=None, target_transform=None, download=True)

# Create the same models for the Fashion MNIST dataset
models_fashion_mnist = [
    DeepSNNModel([28*28, 256, 128, 10]),
    DeepSNNModel([28*28, 256, 128, 64, 10]),
    DeepSNNModel([28*28, 256, 128, 10], recurrent=True),
    DeepSNNModel([28*28, 256, 128, 64, 10], recurrent=True)
]

# Standardize the data
fmnist_x_train = np.array(train_dataset.data, dtype=np.float)
fmnist_x_train = fmnist_x_train.reshape(fmnist_x_train.shape[0], -1) / 255
fmnist_x_test = np.array(test_dataset.data, dtype=np.float)
fmnist_x_test = fmnist_x_test.reshape(fmnist_x_test.shape[0], -1) / 255

fmnist_y_train = np.array(train_dataset.targets, dtype=np.int)
fmnist_y_test = np.array(test_dataset.targets, dtype=np.int)

fashion_mnist_stats = []
i = 1
for i, model in enumerate(models_fashion_mnist):
    model.train(mnist_x_train, mnist_y_train, 256, num_epochs=30) # train the model

    # Get metrics
    accuracy, precision, recall, f1, confusion_matrix = get_metrics(mnist_x_test, mnist_y_test, 256, model)

    # Create dictionary for later use
    stats = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'confusion_matrix': confusion_matrix
    }

    # Print and save the stats
    print(f'Model {i+1}: {stats}')
    fashion_mnist_stats.append(stats)

11.6%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

35.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

58.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

82.8%IOPub

Extracting cached_datasets\FashionMNIST\raw\train-images-idx3-ubyte.gz to cached_datasets\FashionMNIST\raw

Extracting cached_datasets\FashionMNIST\raw\train-labels-idx1-ubyte.gz to cached_datasets\FashionMNIST\raw

Extracting cached_datasets\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to cached_datasets\FashionMNIST\raw

Processing...
Done!


KeyboardInterrupt: 

In [48]:
mnist_df = pd.DataFrame({
    'model': [f'{i}' for i in range(1, len(models_mnist) + 1)],
    'accuracy': [x['accuracy'] for x in mnist_stats],
    'precision': [x['precision'] for x in mnist_stats],
    'recall': [x['recall'] for x in mnist_stats],
    'f1': [x['f1'] for x in mnist_stats],
})

mnist_df

,model,accuracy,precision,recall,f1
0,1,0.969752,0.969747,0.969537,0.969496
1,2,0.963742,0.963730,0.963078,0.963299
2,3,0.963742,0.964089,0.963182,0.963419
3,4,0.920373,0.923151,0.919851,0.920279


In [49]:
# Print confusion matrix for each model
print('Confusion Matrices:')
for confusion_matrix in map(lambda x: x['confusion_matrix'], mnist_stats):
    print(confusion_matrix)

Confusion Matrices:
[[ 972    0    0    2    0    0    1    0    2    2]
 [   3 1115    3    1    2    1    4    1    3    0]
 [  10    0  991    5    3    0    1    6   13    1]
 [   1    0    7  975    0    7    0    7   10    1]
 [   0    0    1    0  969    0    3    1    1    5]
 [   4    0    0   17    1  847    9    2    9    1]
 [   9    2    0    0    4    2  937    1    1    0]
 [   3    8    6    3    1    0    1  989    8    8]
 [  12    1    2    3    6    2    4    2  936    5]
 [  10    3    0    5   25    4    0    3    7  951]]
[[ 968    0    0    0    1    0    2    1    5    2]
 [   1 1118    3    1    0    0    3    1    6    0]
 [   5    1  992    8    2    0    3    8    9    2]
 [   1    1    5  979    0    3    0    7    9    3]
 [   0    3    5    0  941    0    3    2    1   25]
 [   6    0    2   28    3  828    8    2    8    5]
 [   6    3    0    1    8   13  915    0    9    1]
 [   1    3   12    5    3    1    0  988    2   12]
 [   4    0    3   10    

In [51]:
# Create output directory to save the results
output_path = 'results'
os.makedirs(output_path, exist_ok=True)


In [52]:
# Save as XLSX (excel) file
mnist_df.to_excel(os.path.join(output_path, 'mnist_results.xlsx'))

In [ ]:
fmnist_df = pd.DataFrame({
    'model': [f'{i}' for i in range(1, len(models_fashion_mnist))],
    'accuracy': [x['accuracy'] for x in fashion_mnist_stats],
    'precision': [x['precision'] for x in fashion_mnist_stats],
    'recall': [x['recall'] for x in fashion_mnist_stats],
    'f1': [x['f1'] for x in fashion_mnist_stats],
})

fmnist_df

In [ ]:
# Print confusion matrix for each model
print('Confusion Matrices:')
for confusion_matrix in map(lambda x: x['confusion_matrix'], fashion_mnist_stats):
    print(confusion_matrix)

In [ ]:
# Save as XLSX
fmnist_df.to_excel(os.path.join(output_path, 'fashion_mnist_results.xlsx'))